# Preprocessing

## Imports

In [26]:
import pandas as pd
import os

## Load datasets

In [28]:
spending_df = pd.read_csv('./datasets/spending_patterns_detailed.csv')

## Spending patterns

### Filter the columns

In [29]:
spending_df = spending_df[['Customer ID', 'Category', 'Transaction Date', 'Total Spent']]
spending_df.rename(columns={
    'Customer ID': 'customer',
    'Category': 'category',
    'Transaction Date': 'date',
    'Total Spent': 'amount'
}, inplace=True)

### Create categories

In [30]:
# Función para seleccionar un único usuario por categoría
def select_user_by_category(df):
    """
    Selecciona un único usuario representativo para cada categoría.
    """
    user_categories = {}

    # Tacaño: Usuario con el gasto total más bajo
    thrifty_user = df.groupby('customer').sum().sort_values('amount').head(1).index[0]
    user_categories['miserly'] = df[df['customer'] == thrifty_user]

    # Estudiante: Usuario con mayor proporción de gastos en 'Groceries' y 'Food'
    student_user = (
        df[df['category'].isin(['Groceries', 'Food'])]
        .groupby('customer').sum()
        .sort_values('amount', ascending=False)
        .head(1).index[0]
    )
    user_categories['student'] = df[df['customer'] == student_user]

    # Derrochador: Usuario con el gasto total más alto
    big_spender = df.groupby('customer').sum().sort_values('amount', ascending=False).head(1).index[0]
    user_categories['wasteful'] = df[df['customer'] == big_spender]

    # Familiar: Usuario con mayor proporción de gastos en 'Housing and Utilities' y 'Groceries'
    family_user = (
        df[df['category'].isin(['Housing and Utilities', 'Groceries'])]
        .groupby('customer').sum()
        .sort_values('amount', ascending=False)
        .head(1).index[0]
    )
    user_categories['familiar'] = df[df['customer'] == family_user]

    # Aventurero: Usuario con mayor proporción de gastos en 'Travel' y 'Hobbies'
    adventurous_user = (
        df[df['category'].isin(['Travel', 'Hobbies'])]
        .groupby('customer').sum()
        .sort_values('amount', ascending=False)
        .head(1).index[0]
    )
    user_categories['adventurous'] = df[df['customer'] == adventurous_user]

    return user_categories

### Saving the final datasets

In [31]:
user_categories = select_user_by_category(spending_df)

output_dir = './datasets/users'
os.makedirs(output_dir, exist_ok=True)

for category, data in user_categories.items():
    data.to_csv(f'{output_dir}/{category.lower()}.csv', index=False)